In [ ]:
import os
import imageio
from PIL import Image
from IPython.display import display

import numpy as np
import torch
import tensorflow as tf

import glob
from google.colab import drive

In [ ]:
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
def unfold(n,A):
  if n==1:
    unfolded_matrix = [];
    for i in range(A.shape[1]):
      unfolded_matrix.append(A[:,i,:])

    return np.hstack(unfolded_matrix)

  if n==2:
    unfolded_matrix = [];
    for i in range(A.shape[2]):
      unfolded_matrix.append(A[:,:,i].T)

    return np.hstack(unfolded_matrix)

  if n==3:
    unfolded_matrix = [];
    for i in range(A.shape[0]):
      unfolded_matrix.append(A[i,:,:].T)

    return np.hstack(unfolded_matrix)

In [ ]:
def fold1(A, n, d):
    m = A.shape[0]
    folded_matrix = np.zeros((m,n,d));
    for i in range(n):
      folded_matrix[:,i,:] = A[:, i*d:(i+1)*d]

    return folded_matrix

def fold2(A, m, d):
    n = A.shape[0]
    folded_matrix = np.zeros((m,n,d));
    for i in range(d):
      folded_matrix[:,:,i] = A[:, i*m:(i+1)*m].T

    return folded_matrix

def fold3(A, m, n):
    d = A.shape[0]
    folded_matrix = np.zeros((m,n,d));
    for i in range(m):
      folded_matrix[i,:,:] = A[:, i*n:(i+1)*n].T

    return folded_matrix

In [ ]:
def multi1(tensor, A):
    unfolded_matrix = unfold(1,tensor)

    return A @ unfolded_matrix

def multi2(tensor, A):
    unfolded_matrix = unfold(2,tensor)

    return A @ unfolded_matrix

def multi3(tensor, A):
    unfolded_matrix = unfold(3,tensor)

    return A @ unfolded_matrix

In [ ]:
A = torch.zeros(1024, 10, 14)

# Loop through each person
for person_index in range(14):
    train_folder_path = '/content/gdrive/My Drive/train'

    image_pattern = f'subject{person_index + 1:02d}.*.gif'
    items = glob.glob(os.path.join(train_folder_path, image_pattern))

    # Resize images to 32x32 and store in A
    c = 0
    for each_image in items:
        full_path = os.path.join(train_folder_path, each_image)
        image = Image.open(full_path).convert('L')  # Open image and convert to grayscale
        resized_image = image.resize((32, 32))
        image_array = np.array(resized_image, dtype='float64')
        A[:, c, person_index] = torch.from_numpy(image_array.ravel()).to(A)
        c += 1


In [ ]:
A

tensor([[[243., 243., 243.,  ..., 243., 243., 243.],
         [243., 243., 243.,  ..., 244., 243., 243.],
         [243., 157., 243.,  ..., 243., 243., 243.],
         ...,
         [243., 242., 243.,  ..., 237., 243., 243.],
         [243., 242., 243.,  ..., 242., 243., 243.],
         [243., 242., 243.,  ..., 237., 243., 243.]],

        [[243., 243., 243.,  ..., 243., 243., 243.],
         [243., 243., 243.,  ..., 244., 243., 243.],
         [243., 158., 243.,  ..., 242., 243., 243.],
         ...,
         [243., 242., 243.,  ..., 242., 243., 243.],
         [243., 242., 243.,  ..., 241., 243., 243.],
         [243., 242., 243.,  ..., 242., 243., 243.]],

        [[243., 243., 243.,  ..., 243., 243., 243.],
         [243., 243., 243.,  ..., 244., 243., 243.],
         [243., 160., 243.,  ..., 242., 243., 243.],
         ...,
         [243., 242., 243.,  ..., 243., 243., 243.],
         [243., 242., 243.,  ..., 241., 243., 243.],
         [243., 242., 243.,  ..., 243., 243., 243.]],

In [ ]:
test_folder_path = '/content/gdrive/My Drive/test'

Test = []

for i in range(1, 15):
    image_name = f"subject{i:02d}.exist.gif"
    image_path = os.path.join(test_folder_path, image_name)

    image = np.array(Image.open(image_path).convert('L').resize((32, 32)), dtype='float64')
    Test.append(image.ravel())

for i in range(1, 12):
    image_name = f"subject15.{i}.not_exist.gif"
    image_path = os.path.join(test_folder_path, image_name)

    image = np.array(Image.open(image_path).convert('L').resize((32, 32)), dtype='float64')
    Test.append(image.ravel())

Test = np.vstack(Test)


print("Test Shape:", Test.shape)


Test Shape: (25, 1024)


In [ ]:
unfold1A = unfold(1,A)
U1, Sigma, Vt = np.linalg.svd(unfold1A)
F = U1[:,:10 * 14]

unfold2A = unfold(2,A)
U2, Sigma, Vt = np.linalg.svd(unfold2A)
G = U2

unfold3A = unfold(3,A)
U3, Sigma, Vt = np.linalg.svd(unfold3A)
H = U3

print("F Shape:", F.shape)
print("G Shape:", G.shape)
print("H Shape:", H.shape)

F Shape: (1024, 140)
G Shape: (10, 10)
H Shape: (14, 14)


In [ ]:
AF = multi1(A, F.T)
AF = fold1(AF, 10, 14)
print("AxF Shape:", AF.shape)


AFG =  multi2(AF, G.T)
AFG = fold2(AFG, 10 * 14, 14)
print("AxFxG Shape:", AFG.shape)

S =  multi3(AFG, H.T)
S = fold3(S, 10 * 14, 10)
print("S Shape:", S.shape)

AxF Shape: (140, 10, 14)
AxFxG Shape: (140, 10, 14)
S Shape: (140, 10, 14)


In [ ]:
B = multi2(S,G)
B = fold2(B, 10 * 14, 14)
B.shape

(140, 10, 14)

In [ ]:
results = []

for test_image in Test:
    Z = F.T @ test_image

    found_match = False
    for e in range(10):
        Be = B[:, e, :]
        Q, R = np.linalg.qr(Be)
        a = np.linalg.solve(R , Q.T @ Z)
        for i in range(14):
            if np.linalg.norm(a - H[i]) < 0.5:
                results.append(i)
                found_match = True
                break

        if found_match:
            break

    if not found_match:
        results.append((None))

updated_results = [result + 1 if result is not None else 15 for result in results]

for i, result in enumerate(updated_results):
    if result != 15:
        print(f"Test image {i+1} matches training image {result}")
    else:
        print(f"Test image {i+1} does not match any training image")



Test image 1 matches training image 1
Test image 2 matches training image 2
Test image 3 matches training image 3
Test image 4 matches training image 4
Test image 5 matches training image 5
Test image 6 matches training image 6
Test image 7 matches training image 7
Test image 8 matches training image 8
Test image 9 does not match any training image
Test image 10 matches training image 3
Test image 11 matches training image 11
Test image 12 matches training image 5
Test image 13 matches training image 13
Test image 14 matches training image 14
Test image 15 does not match any training image
Test image 16 matches training image 10
Test image 17 does not match any training image
Test image 18 does not match any training image
Test image 19 does not match any training image
Test image 20 does not match any training image
Test image 21 does not match any training image
Test image 22 does not match any training image
Test image 23 does not match any training image
Test image 24 does not matc

In [ ]:
print(updated_results)

[1, 2, 3, 4, 5, 6, 7, 8, 15, 3, 11, 5, 13, 14, 15, 10, 15, 15, 15, 15, 15, 15, 15, 15, 15]


In [ ]:
y_true=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,15,15,15,15,15,15,15,15,15,15]

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_true, updated_results)

0.84